In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()
output_null = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./info.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_ = input_[input_['Kit_Part_Number'] != ''].drop_duplicates(['Kit_JOIN_TCPK'],
                                                                 keep='first',
                                                                 ignore_index=True)

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error, output_null
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                url = 'https://www.rockauto.com/en/parts/' + input_.loc[a, 'Kit_Manufacturer'].replace('+', '%2B').replace(',', '%2C') + ',' + input_.loc[a, 'Kit_Part_Number'] + ',Lennon,' + input_.loc[a, 'Kit_Type_Code']

                resp = requests.get(url,
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text

                # = = = = = = = = = = = = = = =

                soup = BeautifulSoup(resp, 'lxml')
                html = etree.HTML(str(soup))
                
                # = = = = = = = = = = = = = = =
                
                manufacturer = html.xpath('//select[@id="manufacturer_partsearch_007"]/option[@selected]/text()')[0].strip()

                if manufacturer != input_.loc[a, 'Kit_Manufacturer']:
                    crawler_status = 'error'
                    break
                    
                # = = = = = = = = = = = = = = =
                
                no_parts_found = html.xpath('//span[@class="navlabellink nvoffset nnormal"]/text()')
                
                # = = = = = = = = = = = = = = =

                if len(no_parts_found) != 0 and no_parts_found[0].strip() == 'No Parts Found':
                    crawler_status = 'null'
                    break
                    
                # = = = = = = = = = = = = = = =
                
                list_part = html.xpath('//tbody[contains(@class, "listing-inner") and (contains(@class, "altrow-a-1") or contains(@class, "altrow-a-0"))]')

                if len(list_part) == 0:
                    continue
                    
                # = = = = = = = = = = = = = = =
                
                list_json = [json.loads(part.xpath('./descendant::input[contains(@id, "listing_data_supplemental")]/@value')[0]) for part in list_part]

                # = = = = = = = = = = = = = = =
                
                list_json_2 = [json.loads(part.xpath('./descendant::input[contains(@id, "listing_data_essential")]/@value')[0]) for part in list_part]

                # = = = = = = = = = = = = = = =

                index_part = -1
                for i in range(len(list_json)):
                    if list_json[i]['catalogname'].strip() == input_.loc[a, 'Kit_Manufacturer'] and list_json[i]['partnumber'].strip() == input_.loc[a, 'Kit_Part_Number'] and list_json_2[i]['parttype'].strip() == input_.loc[a, 'Kit_Type_Code']:
                        if index_part != -1:
                            continue
                        index_part = i

                if index_part == -1:
                    crawler_status == 'error'
                    break

                # = = = = = = = = = = = = = = =

                category = html.xpath('//select[@id="partgroup_partsearch_007"]/option[@selected]/text()')[0].strip()

                # = = = = = = = = = = = = = = =
                
                type_ = html.xpath('//select[@id="parttype_partsearch_007"]/option[@selected]/text()')[0].strip()

                # = = = = = = = = = = = = = = =
                
                list_note_1 = list_part[index_part].xpath('./descendant::div[contains(@class, "listing-text-row-moreinfo-truck")]/span[contains(@class, "span-link-underline-remover")]/text()')
                note_1 = '' if len(list_note_1) == 0 else ';'.join([note_1.strip() for note_1 in list_note_1])

                # = = = = = = = = = = = = = = =
                
                list_note_2 = list_part[index_part].xpath('./descendant::span[contains(@title, "Please make sure")]/span/text()')
                note_2 = '' if len(list_note_2) == 0 else ';'.join([note_2.strip() for note_2 in list_note_2])

                # = = = = = = = = = = = = = = =

                src = list_part[index_part].xpath('./descendant::img[contains(@id, "inlineimg_thumb")]')
                src = '' if len(src) == 0 else 'https://www.rockauto.com' + src[0].xpath('./@src')[0].strip().replace('__ra_m.', '.')

                # = = = = = = = = = = = = = = =
                
                oe = list_part[index_part].xpath('./descendant::span[contains(@title, "Replaces these Alternate")]')
                oe = '' if len(oe) == 0 else oe[0].xpath('./text()')[0].strip().replace(', ', ';')
                oe = ';'.join(sorted(list(set(oe.split(';'))), key=oe.split(';').index))
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'Kit_JOIN_TCPK': input_.loc[a, 'Kit_JOIN_TCPK'],
                                         'Kit_Category': category,
                                         'Kit_Type': type_,
                                         'Kit_Vehcile_Json': '',
                                         'Kit_Vehcile': '',
                                         'Kit_OE': oe,
                                         'Kit_Note_1': note_1,
                                         'Kit_Note_2': note_2,
                                         'Kit_Pic': '',
                                         'Kit_Url': url,
                                         'Kit_Info': 'https://www.rockauto.com/en/moreinfo.php?pt=' + input_.loc[a, 'Kit_Type_Code'] + '&pk=' + input_.loc[a, 'Kit_Part_Key'],
                                         'Kit_Src': src,
                                         'Kit_Specifications_Json': '',
                                         'Kit_Kit_Json': ''}])

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Info': input_.loc[a, 'Info'],
                                     'Kit_No.': int(input_.loc[a, 'Kit_No.']),
                                     'Kit_Quantity': int(input_.loc[a, 'Kit_Quantity']),
                                     'Kit_Note': input_.loc[a, 'Kit_Note'],
                                     'Kit_Manufacturer': input_.loc[a, 'Kit_Manufacturer'],
                                     'Kit_Part_Number': input_.loc[a, 'Kit_Part_Number'],
                                     'Kit_Type_Code': input_.loc[a, 'Kit_Type_Code'],
                                     'Kit_Part_Key': input_.loc[a, 'Kit_Part_Key'],
                                     'Kit_JOIN_TCPK': input_.loc[a, 'Kit_JOIN_TCPK']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'null':
            df_temp = pd.DataFrame([{'Kit_JOIN_TCPK': input_.loc[a, 'Kit_JOIN_TCPK'],
                                     'Kit_Category': '',
                                     'Kit_Type': '',
                                     'Kit_Vehcile_Json': '',
                                     'Kit_Vehcile': '',
                                     'Kit_OE': '',
                                     'Kit_Note_1': '',
                                     'Kit_Note_2': '',
                                     'Kit_Pic': '',
                                     'Kit_Url': '',
                                     'Kit_Info': 'https://www.rockauto.com/en/moreinfo.php?pt=' + input_.loc[a, 'Kit_Type_Code'] + '&pk=' + input_.loc[a, 'Kit_Part_Key'],
                                     'Kit_Src': '',
                                     'Kit_Specifications_Json': '',
                                     'Kit_Kit_Json': ''}])

            output_null = pd.concat([output_null, df_temp], ignore_index=True).fillna('')
    
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Kit_JOIN_TCPK'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Kit_JOIN_TCPK'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./kit_part-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Kit_JOIN_TCPK'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./kit_part_error.xlsx', index=False)
    print('爬虫存在error')
    print()
if len(output_null) != 0:
    output_null = output_null.drop_duplicates(ignore_index=True)
    output_null = output_null.sort_values(by=['Kit_JOIN_TCPK'], ascending=[True]).reset_index(drop=True)
    output_null.to_excel('./kit_part_null-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
    print('爬虫存在null')
    print()
print('Done ~')

总数量：926

[ok] - 5736;8307904
[尝试次数：7] - [剩余数量：906] - [当前时间：09:16:37]

[ok] - 5724;10110008
[尝试次数：9] - [剩余数量：905] - [当前时间：09:16:38]

[ok] - 5736;10110028
[尝试次数：12] - [剩余数量：904] - [当前时间：09:16:39]

[ok] - 5736;8307912
[尝试次数：18] - [剩余数量：903] - [当前时间：09:16:42]

[ok] - 5723;8314716
[尝试次数：27] - [剩余数量：902] - [当前时间：09:16:44]

[ok] - 10465;10195124
[尝试次数：25] - [剩余数量：901] - [当前时间：09:16:44]

[ok] - 5724;7093400
[尝试次数：28] - [剩余数量：900] - [当前时间：09:16:45]

[ok] - 5728;10519392
[尝试次数：13] - [剩余数量：899] - [当前时间：09:16:45]

[ok] - 5728;10519400
[尝试次数：2] - [剩余数量：898] - [当前时间：09:16:46]

[ok] - 5723;8314756
[尝试次数：28] - [剩余数量：897] - [当前时间：09:16:46]

[ok] - 5724;10195068
[尝试次数：34] - [剩余数量：896] - [当前时间：09:16:47]

[ok] - 5724;10454488
[尝试次数：1] - [剩余数量：895] - [当前时间：09:16:48]

[ok] - 5728;10195092
[尝试次数：40] - [剩余数量：894] - [当前时间：09:16:49]

[ok] - 5722;10195128
[尝试次数：21] - [剩余数量：893] - [当前时间：09:16:49]

[ok] - 5736;10454552
[尝试次数：1] - [剩余数量：892] - [当前时间：09:16:49]

[ok] - 5736;10110036
[尝试次数：39] - [剩余数量：891] - [当前时间：09:

[ok] - 5568;8527968
[尝试次数：1] - [剩余数量：773] - [当前时间：09:18:15]

[ok] - 5722;8313608
[尝试次数：1] - [剩余数量：772] - [当前时间：09:18:16]

[ok] - 5728;8307896
[尝试次数：2] - [剩余数量：771] - [当前时间：09:18:16]

[ok] - 5728;8308272
[尝试次数：3] - [剩余数量：770] - [当前时间：09:18:16]

[ok] - 11860;11158181
[尝试次数：1] - [剩余数量：769] - [当前时间：09:18:17]

[ok] - 5728;11158185
[尝试次数：1] - [剩余数量：768] - [当前时间：09:18:17]

[ok] - 5736;11158189
[尝试次数：1] - [剩余数量：767] - [当前时间：09:18:17]

[ok] - 5728;11158165
[尝试次数：2] - [剩余数量：766] - [当前时间：09:18:18]

[ok] - 5723;11158217
[尝试次数：1] - [剩余数量：765] - [当前时间：09:18:18]

[ok] - 5723;11158213
[尝试次数：1] - [剩余数量：764] - [当前时间：09:18:18]

[ok] - 5736;11158193
[尝试次数：1] - [剩余数量：763] - [当前时间：09:18:19]

[ok] - 5723;10890380
[尝试次数：6] - [剩余数量：762] - [当前时间：09:18:19]

[ok] - 18586;8317256
[尝试次数：1] - [剩余数量：761] - [当前时间：09:18:19]

[ok] - 5728;10890324
[尝试次数：10] - [剩余数量：760] - [当前时间：09:18:19]

[ok] - 5589;11158221
[尝试次数：1] - [剩余数量：759] - [当前时间：09:18:19]

[ok] - 5728;8307276
[尝试次数：2] - [剩余数量：758] - [当前时间：09:18:20]

[ok] - 5736

[ok] - 5724;14047605
[尝试次数：3] - [剩余数量：640] - [当前时间：09:19:16]

[ok] - 5724;8306424
[尝试次数：13] - [剩余数量：639] - [当前时间：09:19:16]

[ok] - 5736;14047813
[尝试次数：5] - [剩余数量：638] - [当前时间：09:19:16]

[ok] - 5728;14047805
[尝试次数：3] - [剩余数量：637] - [当前时间：09:19:17]

[ok] - 5736;14047837
[尝试次数：3] - [剩余数量：636] - [当前时间：09:19:17]

[ok] - 5728;14047849
[尝试次数：1] - [剩余数量：635] - [当前时间：09:19:17]

[ok] - 5736;14047841
[尝试次数：1] - [剩余数量：634] - [当前时间：09:19:18]

[ok] - 5724;14047609
[尝试次数：1] - [剩余数量：633] - [当前时间：09:19:18]

[ok] - 5728;14047829
[尝试次数：10] - [剩余数量：632] - [当前时间：09:19:19]

[ok] - 5728;13570105
[尝试次数：1] - [剩余数量：631] - [当前时间：09:19:19]

[ok] - 5723;14047949
[尝试次数：3] - [剩余数量：630] - [当前时间：09:19:20]

[ok] - 5589;12804309
[尝试次数：6] - [剩余数量：629] - [当前时间：09:19:23]

[ok] - 5723;14047945
[尝试次数：1] - [剩余数量：628] - [当前时间：09:19:23]

[ok] - 5722;8314836
[尝试次数：7] - [剩余数量：627] - [当前时间：09:19:24]

[ok] - 5568;13570021
[尝试次数：9] - [剩余数量：626] - [当前时间：09:19:24]

[ok] - 5728;14047845
[尝试次数：4] - [剩余数量：625] - [当前时间：09:19:24]

[ok] - 5

[ok] - 12910;15583717
[尝试次数：1] - [剩余数量：508] - [当前时间：09:20:26]

[ok] - 5728;15383237
[尝试次数：12] - [剩余数量：507] - [当前时间：09:20:27]

[ok] - 5564;10981408
[尝试次数：2] - [剩余数量：506] - [当前时间：09:20:27]

[ok] - 5724;8304548
[尝试次数：2] - [剩余数量：505] - [当前时间：09:20:27]

[ok] - 5728;15383277
[尝试次数：3] - [剩余数量：504] - [当前时间：09:20:28]

[ok] - 57235;15383909
[尝试次数：9] - [剩余数量：503] - [当前时间：09:20:29]

[ok] - 5728;8308240
[尝试次数：12] - [剩余数量：502] - [当前时间：09:20:29]

[ok] - 5728;15383281
[尝试次数：4] - [剩余数量：501] - [当前时间：09:20:29]

[ok] - 5723;9659232
[尝试次数：5] - [剩余数量：500] - [当前时间：09:20:30]

[ok] - 5728;15383313
[尝试次数：8] - [剩余数量：499] - [当前时间：09:20:30]

[ok] - 18586;8317280
[尝试次数：9] - [剩余数量：498] - [当前时间：09:20:31]

[ok] - 5724;15583565
[尝试次数：10] - [剩余数量：497] - [当前时间：09:20:32]

[ok] - 5568;8304552
[尝试次数：24] - [剩余数量：496] - [当前时间：09:20:33]

[ok] - 10709;14047821
[尝试次数：12] - [剩余数量：495] - [当前时间：09:20:33]

[ok] - 5724;8306220
[尝试次数：1] - [剩余数量：494] - [当前时间：09:20:34]

[ok] - 5724;8305700
[尝试次数：8] - [剩余数量：493] - [当前时间：09:20:35]

[ok] -